In [ ]:
################################################################################
# INSTALL CONDA ON GOOGLE COLAB
################################################################################
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2022-06-21 02:55:49--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   132MB/s    in 0.6s    

2022-06-21 02:55:50 (132 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h7b64

In [ ]:
! nvidia-smi

Tue Jun 21 02:56:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! conda create -n rapids-22.04 -c rapidsai -c nvidia -c conda-forge  \
    cuml=22.04 python=3.9 cudatoolkit=11.5

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

# New section

In [ ]:
print("hello")

hello


In [ ]:
import pandas as pd
df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

print("hmm")
print(len(set(df_harvey["Text"].tolist())))
print(len(set(df_sandy["Text"].tolist())))
print(len(set(df_michael["Text"].tolist())))
print(len(set(df_florence["Text"].tolist())))
print(len(set(df_all["Text"].tolist())))

print(len(set(english_text)))



229455
230813
810768
982234
2253270
hmm
102904
85322
696225
862595
1742940
1742940


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
pip install -U kaleido
pip install gensim
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

num_topics = 70

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False, nr_topics=num_topics)
topics, probs = model.fit_transform(list(set(english_text)))

print(model.get_topic_info())

result = model.get_topic_info()

info_details = []
for x in range(len(result)):
    info_details.append(model.get_topic(x))

result["Topics"] = info_details

result.to_csv("bertopic_70.csv")
print("csv saved")
# fig = topic_model.visualize_topics()
# fig.write_image
# fig2 = model.visualize_barchart()
# fig2.show()
# fig3 = model.visualize_heatmap()
# fig3.show()

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

vectorizer = model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [tokenizer(doc) for doc in english_text]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

print("coherence")
print(coherence)

model.save("topic70")
print("model saved")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Label prop iterations: 59
Label prop iterations: 18
Label prop iterations: 14
Label prop iterations: 9
Label prop iterations: 5
Label prop iterations: 5
Iterations: 6
25893,414,40970,19,1011,75420
Label prop iterations: 2
Iterations: 1
1031,130,5953,6,183,222
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Python version
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]
229455
230813
810768
9822

tcmalloc: large alloc 2677161984 bytes == 0x555a67976000 @  0x7f8e820521e7 0x7f8e805b9655 0x7f8e805b9ba0 0x7f8e80600ea0 0x7f8e806039d4 0x7f8e80603ed7 0x7f8e8069baee 0x7f8e8069bf22 0x5558d738e9c7 0x5558d738b636 0x5558d7385a17 0x5558d73a7328 0x5558d7387bcb 0x5558d7385a17 0x5558d7397ff9 0x5558d7387063 0x5558d7385a17 0x5558d7397ff9 0x5558d7387063 0x5558d7385a17 0x5558d73a7328 0x5558d7387bcb 0x5558d7385a17 0x5558d7397ff9 0x5558d7387063 0x5558d7385a17 0x5558d73856d7 0x5558d7385689 0x5558d7440e3b 0x5558d746f4a9 0x5558d746b694
tcmalloc: large alloc 1215234048 bytes == 0x555b52a10000 @  0x7f8e820521e7 0x7f8e805b9655 0x7f8e805b9ba0 0x7f8e80600ea0 0x7f8e806019b7 0x7f8e80698e70 0x5558d738e9c7 0x5558d7387bcb 0x5558d7398303 0x5558d73a7203 0x5558d738b636 0x5558d7398303 0x5558d73a7203 0x5558d738b636 0x5558d7398303 0x5558d7411b89 0x5558d74b860f 0x5558d73aac4a 0x5558d744aa1a 0x5558d738a17d 0x5558d7385a17 0x5558d7397ff9 0x5558d7387bcb 0x5558d7385a17 0x5558d7397ff9 0x5558d7386db3 0x5558d7385a17 0x5558d739

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
pip install -U kaleido
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

model = BERTopic.load("/content/drive/My Drive/for colab/topic30")
fig = model.visualize_topics()
fig.show()
fig.write_image("30_intertopic.png")
fig2 = model.visualize_barchart()
fig2.write_image("30_bar.png")
fig3 = model.visualize_heatmap()
fig3.write_image("30_heatmap.png")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python version
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]


tcmalloc: large alloc 2677161984 bytes == 0x55d78b80a000 @  0x7fbd9c4b61e7 0x7fbd9a010655 0x7fbd9a010ba0 0x7fbd9a057ea0 0x7fbd9a0589b7 0x7fbd9a0efe70 0x55d7101279c7 0x55d710120bcb 0x55d71011ea17 0x55d710130ff9 0x55d710120063 0x55d710131303 0x55d710120063 0x55d710131303 0x55d71011fdb3 0x55d710131303 0x55d710120063 0x55d71011ea17 0x55d710130ff9 0x55d71011fdb3 0x55d71011ea17 0x55d710130ff9 0x55d710124636 0x55d71011ea17 0x55d710140328 0x55d710124636 0x55d71011ea17 0x55d71011e6d7 0x55d71011e689 0x55d7101d9e3b 0x55d7102084a9


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
pip install -U kaleido
pip install gensim
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

model = BERTopic.load("/content/drive/My Drive/for colab/topic250f")

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

vectorizer = model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [tokenizer(doc) for doc in english_text]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

print(coherence)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Python version
3.9.13 | packaged by conda-forge | (main, May 27 2022, 16:56:21) 
[GCC 10.3.0]
229455
230813
810768
982234
2253270


tcmalloc: large alloc 2677161984 bytes == 0x561edda5c000 @  0x7f0331d3e1e7 0x7f032f898655 0x7f032f898ba0 0x7f032f8dfea0 0x7f032f8e09b7 0x7f032f977e70 0x561e839959c7 0x561e8398ebcb 0x561e8398ca17 0x561e8399eff9 0x561e8398e063 0x561e8399f303 0x561e8398e063 0x561e8399f303 0x561e8398ddb3 0x561e8399f303 0x561e8398e063 0x561e8398ca17 0x561e8399eff9 0x561e8398ddb3 0x561e8398ca17 0x561e8399eff9 0x561e83992636 0x561e8398ca17 0x561e839ae328 0x561e83992636 0x561e8398ca17 0x561e8398c6d7 0x561e8398c689 0x561e83a47e3b 0x561e83a764a9
Traceback (most recent call last):
  File "<stdin>", line 40, in <module>
NameError: name 'topics' is not defined


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

num_topics = 150

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
topics, probs = model.fit_transform(english_text)

print(model.get_topic_info())

result = model.get_topic_info()

info_details = []
for x in range(len(result)):
    info_details.append(model.get_topic(x))

result["Topics"] = info_details

result.to_csv("bertopic_150.csv")
model.save("topic150")

Process is terminated.


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

num_topics = 100

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
topics, probs = model.fit_transform(english_text)

print(model.get_topic_info())

result = model.get_topic_info()

info_details = []
for x in range(len(result)):
    info_details.append(model.get_topic(x))

result["Topics"] = info_details

result.to_csv("bertopic_100.csv")
model.save("topic100")

Process is interrupted.


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

num_topics = 50

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
topics, probs = model.fit_transform(english_text)

print(model.get_topic_info())

result = model.get_topic_info()

info_details = []
for x in range(len(result)):
    info_details.append(model.get_topic(x))

result["Topics"] = info_details

result.to_csv("bertopic_50.csv")
model.save("topic50")

Process is interrupted.


In [ ]:
# activate your conda environment
%%bash
source activate rapids-22.04
pip install bertopic
python

# python commands are ready to run within your environment
import sys
print("Python version")
print (sys.version)
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from bertopic import BERTopic
import numpy as np
import pandas as pd

df_harvey = pd.read_csv("harvey_clean.csv")
df_sandy = pd.read_csv("sandy_clean.csv")
df_michael = pd.read_csv("michael_clean.csv")
df_florence = pd.read_csv("florence_clean.csv")
df_all = pd.concat([df_harvey,df_sandy,df_michael,df_florence], ignore_index=True)
print(len(df_harvey))
print(len(df_sandy))
print(len(df_michael))
print(len(df_florence))
print(len(df_all))
english_text = df_all["Text"].tolist()

num_topics = 10

# Create instances of GPU-accelerated UMAP and HDBSCAN
umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True)

# Pass the above models to be used in BERTopic
model = BERTopic(umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=False)
topics, probs = model.fit_transform(english_text)

print(model.get_topic_info())

result = model.get_topic_info()

info_details = []
for x in range(len(result)):
    info_details.append(model.get_topic(x))

result["Topics"] = info_details

result.to_csv("bertopic_10.csv")
model.save("topic10")

Process is interrupted.
